In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
#from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_curve, auc

from sklearn import tree
import graphviz 

In [18]:
#Read the preprocessed One hot data
car_feature = pd.read_csv('oneHotPreprocess.csv')
car_label= car_feature['Lable']
car_feature.drop(['Lable'],inplace=True,axis=1)
#change the column name
car_label= pd.DataFrame(car_label)
print(type(car_label))
lableName= ['Lable']
car_label.columns= lableName

#Read the MwMote generated Data
mwmote_carFeature= pd.read_csv('mwmote_oneHotcar_Feature.csv')
mwmote_carLabel= pd.read_csv('mwmoteGeneratedLabel.csv')

#change the lable name
lableName= ['Lable']
mwmote_carLabel.columns= lableName

mwmote_carLabel= mwmote_carLabel.replace(-1,1)


<class 'pandas.core.frame.DataFrame'>


In [19]:
#combinet both the data
columnName= list(car_feature.columns.values)
combine_car_features= pd.DataFrame(columns=columnName)
combine_car_label= pd.DataFrame(columns=lableName)

i= 0
j= 0
k = 0
totalLength= car_feature.shape[0] + mwmote_carFeature.shape[0]

for i in range(totalLength):
    if i<car_feature.shape[0]:
        combine_car_features.loc[j]= car_feature.loc[i]
        combine_car_label.loc[j]= car_label.loc[i]
        j = j + 1
    else:
        combine_car_features.loc[j]= mwmote_carFeature.loc[k]
        combine_car_label.loc[j]= mwmote_carLabel.loc[k]
        j = j + 1
        k = k + 1
        

'''
feature_frame1= [car_feature,mwmote_carFeature]
lable_frame= [mwmote_carLabel,car_label]

combine_car_features= pd.concat(feature_frame1)
combine_car_lable= pd.concat(lable_frame)

combine_car_features= pd.DataFrame(combine_car_features)
combine_car_lable= pd.DataFrame(combine_car_lable)
'''

'\nfeature_frame1= [car_feature,mwmote_carFeature]\nlable_frame= [mwmote_carLabel,car_label]\n\ncombine_car_features= pd.concat(feature_frame1)\ncombine_car_lable= pd.concat(lable_frame)\n\ncombine_car_features= pd.DataFrame(combine_car_features)\ncombine_car_lable= pd.DataFrame(combine_car_lable)\n'

In [20]:
combine_car_features.head()


,Make_Accura,Make_BMW,Make_Chevrolet,Make_Dodge,Make_Ferrari,Make_Ford,Make_Honda,Make_Jaguar,Make_Lexus,Make_Mazda,...,NumberOfCars_3 to 4,NumberOfCars_5 to 8,NumberOfCars_more than 8,BasePolicy_All Perils,BasePolicy_Collision,BasePolicy_Liability,Age,Deductible,DriverRating,DaysDiff
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.2625,0.000000,0.000000,0.071795
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.4250,1.000000,1.000000,0.069231
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.5875,0.666667,0.666667,0.089744
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.8125,0.333333,0.333333,0.107692
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.3375,0.000000,0.000000,0.094872


In [21]:
combine_car_features.shape

combine_car_label

,Lable
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,0.0
8,0.0
9,0.0


In [22]:
#add columns to existing dataframe


combine_car_features['Label']=combine_car_label 



#shuffle the data.
new_combine_features = combine_car_features.set_index(np.random.permutation(combine_car_features.index))

#now divide the feature and label
new_label = new_combine_features['Label']

#drop the Fraud Found lable
#drop the following attributes
new_combine_features.drop(['Label'],inplace=True,axis=1)


In [23]:
new_combine_features.to_csv('oneHotfinalMwoteDataWithOriginalData.csv',index=None,sep=' ')

In [24]:
#train the model
X_train,X_test,y_train,y_test = train_test_split(new_combine_features,new_label,random_state=3,test_size=0.25)

In [25]:
#for drawing the tree
featureName= list(new_combine_features.columns.values)
lableName= ['Non-Fraudulent','Fraudulent']


In [26]:
#initialize the model
classification_tree= tree.DecisionTreeClassifier()

In [27]:
classification_tree= classification_tree.fit(X_train,y_train)



In [32]:
dot_data = tree.export_graphviz(classification_tree,max_depth=3,out_file=None, 
                     feature_names=featureName,  
                     class_names=lableName,  
                     filled=True, rounded=True,  
                     special_characters=False)

In [33]:
graph = graphviz.Source(dot_data)  
graph.render("carClaimsOneHotDepth3")

'carClaimsOneHotDepth3.pdf'